## Домашнее задание к лекции "Базовые понятия статистики"

### Задание 1. Базовое изучение

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
names = ['surgery','age','hospital_number','rectal_temperature','pulse','respiratory_rate','temperature_of_extremities',
         'peripheral_pulse','mucous_membranes','capillary_refill_time','pain','peristalsis','abdominal_distension',
         'nasogastric_tube','nasogastric reflux','nasogastric_reflux_PH','rectal_examination_feces','abdomen','packed_cell_volume',
         'total_protein','abdominocentesis_appearance','abdomcentesis_total_protein','outcome','surgical_lesion',
         'type_of_lesion_1','type_of_lesion_2','type_of_lesion_3','cp_data'
]
horses = pd.read_csv('files/horse_data.csv', names=names, na_values='?')

Отбросим столбцы с самым большим количеством пропусков, из оставшихся выберем наиболее взаимосвязанные друг с другом:

In [3]:
df = horses[
    ['surgery','age', 'rectal_temperature', 'pulse', 'peripheral_pulse', 'abdominal_distension', 'peristalsis',
     'pain', 'packed_cell_volume', 'outcome', 'surgical_lesion']
]

#### Категориальные признаки
Столбцы **surgery, age, peripheral pulse, abdominal distension, peristalsis, pain, outcome** и **surgical lesion** - категориальные, для них полезно будет рассчитать моду


In [4]:
df_mode = pd.DataFrame()

list_of_attr = ['surgery','age','peripheral_pulse','abdominal_distension','peristalsis','pain','outcome','surgical_lesion']

for attr in list_of_attr:
    new_row = {'Attribute': attr, 'Mode': df[attr].mode()[0]}
    df_mode = pd.concat([df_mode, pd.DataFrame([new_row])])

df_mode

,Attribute,Mode
0,surgery,1.0
0,age,1.0
0,peripheral_pulse,1.0
0,abdominal_distension,1.0
0,peristalsis,3.0
0,pain,3.0
0,outcome,1.0
0,surgical_lesion,1.0


**Выводы** о большинстве больных лошадей:  
1. они подвергались оперативному лечению
1. болеют взрослые животные
1. пульс нормальный
1. вздутия живота не наблюдается
1. перистальтика снижена
1. испытывают боли средней силы
1. результат лечения в основном благоприятный

#### Числовые признаки

Столбцы **rectal_temperature, pulse, packed_cell_volume** - числовые, для них можем рассчитать метрики с помощью метода *describe*

In [5]:
df[['rectal_temperature', 'pulse', 'packed_cell_volume']].describe()

,rectal_temperature,pulse,packed_cell_volume
count,240.000000,276.000000,271.000000
mean,38.167917,71.913043,46.295203
std,0.732289,28.630557,10.419335
min,35.400000,30.000000,23.000000
25%,37.800000,48.000000,38.000000
50%,38.200000,64.000000,45.000000
75%,38.500000,88.000000,52.000000
max,40.800000,184.000000,75.000000


50-й процентиль соответствует медианному значению.

**Выводы:**  
1. Стандартное отклонение температуры невелико - температура большинства больных лошадей повышена незначительно, в среднем выше нормы на 0.3 градуса.
1. Размах пульса более существенен - необходимо проанализировать выбросы в 30 и 184 уд./мин. на предмет их корректности
1. Среднее значение гематокрита у лошадей - на верхней границе нормы (30-50 единиц)

### Задание 2. Работа с выбросами

Изучим выбросы в статистике пульса (минимальное значение 30 и максимальное 184 уд./мин. на предмет возможных ошибок

In [6]:
df[df.pulse > df.pulse.quantile([0.75]).values[0]].sort_values('pulse', ascending=False).head(5)

,surgery,age,rectal_temperature,pulse,peripheral_pulse,abdominal_distension,peristalsis,pain,packed_cell_volume,outcome,surgical_lesion
255,1.0,9,38.8,184.0,NaN,3.0,1.0,4.0,33.0,2.0,1
3,1.0,9,39.1,164.0,1.0,4.0,4.0,2.0,48.0,2.0,1
55,1.0,9,38.6,160.0,NaN,4.0,3.0,3.0,38.0,2.0,1
275,1.0,9,38.8,150.0,3.0,2.0,3.0,5.0,50.0,2.0,1
41,2.0,9,39.0,150.0,NaN,NaN,NaN,NaN,47.0,1.0,1


**Вывод**: максимальной пульс наблюдается у молодой лошади и, вероятно, не является ошибкой: у некоторых других лошадей пульс ненамного ниже максимального. Установлены дополнительные сходства у пяти лошадей с высоким пульсом: животные испытывали сильные боли, лечение завершилось летальным исходом.  
Эти значения корректны и их нужно оставить.

In [7]:
df[df.pulse < df.pulse.quantile([0.25]).values[0]].sort_values('pulse').head(5)

,surgery,age,rectal_temperature,pulse,peripheral_pulse,abdominal_distension,peristalsis,pain,packed_cell_volume,outcome,surgical_lesion
232,1.0,1,38.5,30.0,NaN,NaN,NaN,NaN,40.0,1.0,1
130,1.0,1,37.8,30.0,NaN,NaN,NaN,NaN,NaN,2.0,1
276,1.0,1,38.0,36.0,1.0,2.0,2.0,4.0,37.0,3.0,2
242,1.0,1,NaN,36.0,1.0,3.0,3.0,5.0,41.0,2.0,1
203,2.0,1,37.2,36.0,1.0,1.0,3.0,2.0,35.0,1.0,2


**Вывод**: наиболее низкие значения пульса, вероятно, являются ошибками. Отсутствуют другие показатели, на основе которых можно установить, истинно или ложно значение пульса, равное 30.  
Эти значения некорректны, а т.к. информации по лечению этих лошадей в целом мало, полагаю, что строки могут быть удалены целиком.

### Задание 3. Работа с пропусками

Статистика пропусков в данных по 300 лошадям

In [8]:
df.isna().mean() * 100

surgery                  0.333333
age                      0.000000
rectal_temperature      20.000000
pulse                    8.000000
peripheral_pulse        23.000000
abdominal_distension    18.666667
peristalsis             14.666667
pain                    18.333333
packed_cell_volume       9.666667
outcome                  0.333333
surgical_lesion          0.000000
dtype: float64

Удаление строк, в которых заполнено менее 7 полей, сохраняет 94% информации, при этом число пропусков значительно сокращается:

In [9]:
df.dropna(thresh=7).isna().mean() * 100

surgery                  0.354610
age                      0.000000
rectal_temperature      17.021277
pulse                    5.319149
peripheral_pulse        18.085106
abdominal_distension    13.475177
peristalsis              9.219858
pain                    13.829787
packed_cell_volume       6.382979
outcome                  0.354610
surgical_lesion          0.000000
dtype: float64

In [10]:
data = df.dropna(thresh=7)
print(f'После удаления строк осталось {round(len(data) / len(df) * 100)}% информации')

После удаления строк осталось 94% информации


#### Восстановление пропусков

Начнем восстанавливать пропуски, опираясь на столбцы, в которых либо нет пропусков, либо их количество минимально.  
Кроме того, в файле horse_data.names есть указания на взаимосвязь некоторых атрибутов, используем это.

- Поле **surgery** связано с **surgical lesion** по описанию, восстановим пропуск

In [11]:
data.surgery.fillna(data.surgical_lesion, inplace=True)

- Проверим связь пульса и возраста животного - видим, что среднее значение пульса у молодых лошадей почти в 2 раза выше, чем у взрослых

In [12]:
data.groupby('age').pulse.describe()['mean']

age
1     67.967480
9    122.285714
Name: mean, dtype: float64

    Заполним пропущенные значения пульса средним по возрастной группе:

In [13]:
data.pulse.fillna(data.groupby('age')['pulse'].transform('mean'), inplace=True)

- Исследуем пропуск в поле **outcome** (исход лечения)

In [14]:
data[data.outcome.isna()]

,surgery,age,rectal_temperature,pulse,peripheral_pulse,abdominal_distension,peristalsis,pain,packed_cell_volume,outcome,surgical_lesion
132,1.0,1,38.0,48.0,1.0,1.0,3.0,4.0,43.0,NaN,1


Сравним с похожими лошадьми:

In [15]:
data[data.pulse.between(45, 50) & (data.pain >= 3) & (data.age == 1) & (data.peristalsis >= 3)]

,surgery,age,rectal_temperature,pulse,peripheral_pulse,abdominal_distension,peristalsis,pain,packed_cell_volume,outcome,surgical_lesion
6,1.0,1,37.9,48.0,1.0,3.0,3.0,3.0,37.0,1.0,1
95,1.0,1,37.3,48.0,1.0,3.0,3.0,3.0,41.0,1.0,1
127,1.0,1,37.4,50.0,3.0,1.0,4.0,4.0,45.0,1.0,1
132,1.0,1,38.0,48.0,1.0,1.0,3.0,4.0,43.0,NaN,1
146,1.0,1,38.2,48.0,3.0,4.0,4.0,3.0,42.0,1.0,1
154,2.0,1,38.8,48.0,1.0,4.0,3.0,3.0,41.0,1.0,1
192,1.0,1,38.2,48.0,1.0,3.0,3.0,3.0,48.0,1.0,1
249,2.0,1,38.2,48.0,NaN,1.0,3.0,3.0,34.0,1.0,2


Скорее всего, лошадь выжила - заполним пропуск в поле **outcome** значением 1

In [16]:
data.outcome.fillna(1, inplace=True)

- Обратимся к столбцу **abdominal distension** (вздутие живота), проверим его взаимосвязь с полем **surgery** - если не было операции, то скорее всего вздутия тоже не было, заполним пропуски согласно этому наблюдению. Остальные значения вздутия заполним модой.


In [17]:
data[(data.surgery == 2)]['abdominal_distension'].value_counts()

1.0    41
2.0    34
3.0    15
4.0    11
Name: abdominal_distension, dtype: int64

In [18]:
data.loc[(data.surgery == 2) & data.abdominal_distension.isna(), 'abdominal_distension'] = 1

In [19]:
data.abdominal_distension.fillna(data.abdominal_distension.mode()[0], inplace=True)

- Кроме того, исследователи указывают на взаимосвязь вздутия живота с болью и перистальтикой, заполним пропуски: для каждого значения вздутия заполним поля **pain** и **peristalsis** модальными значениями

In [20]:
def fillna_on_abdominal_dist(row, col):
    '''
    Проверка x == x возвращает False в случае NaN
    '''
    if (row.abdominal_distension == row.abdominal_distension) and (row[col] != row[col]):
        return data[data.abdominal_distension == row.abdominal_distension][col].mode()[0]
    else:
        return row[col]

data['pain'] = data.apply(fillna_on_abdominal_dist, col='pain', axis=1)
data['peristalsis'] = data.apply(fillna_on_abdominal_dist, col='peristalsis', axis=1)


- Так как стандартное отклонение температуры невелико, можно заполнить пропуски средним арифметическим

In [21]:
data.rectal_temperature.fillna(data.rectal_temperature.mean(), inplace=True)

- Существует связь между гематокритом (**packed cell volume**) и исходом лечения (**outcome**): у умерших лошадей он превышает норму. Заполним пропуски:

In [22]:
print('Для выживших лошадей среднее значение гематокрита', round(data[data.outcome == 1].packed_cell_volume.mean(), 2))
print('Для умерших лошадей среднее значение гематокрита', round(data[data.outcome > 1].packed_cell_volume.mean(), 2))

Для выживших лошадей среднее значение гематокрита 42.8
Для умерших лошадей среднее значение гематокрита 52.17


In [23]:
data.packed_cell_volume.fillna(data.groupby('outcome')['packed_cell_volume'].transform('median'), inplace=True)

- Наконец заполним пропуски в значениях периферийного пульса согласно закономерности:

In [24]:
s = data.groupby('peripheral_pulse')['pulse'].mean()
low_pulse = s.values[0]
high_pulse = s.values[2]


In [25]:
# проставим значение 1 в строках с низким пульсом (сравниваю, к чему ближе значение пульса - к нижнему или верхнему среднему)
data.loc[data.peripheral_pulse.isna() & (abs(data.pulse - low_pulse) < abs(data.pulse - high_pulse)), 'peripheral_pulse'] = 1

In [26]:
# проставим значение 3 в строках с высоким пульсом
data.loc[data.peripheral_pulse.isna() & (abs(data.pulse - low_pulse) >= abs(data.pulse - high_pulse)), 'peripheral_pulse'] = 3

#### Итоговый датафрейм не содержит пропусков

In [27]:
data.isna().mean() * 100

surgery                 0.0
age                     0.0
rectal_temperature      0.0
pulse                   0.0
peripheral_pulse        0.0
abdominal_distension    0.0
peristalsis             0.0
pain                    0.0
packed_cell_volume      0.0
outcome                 0.0
surgical_lesion         0.0
dtype: float64